In [16]:
import requests
import fake_useragent
from typing import List
import pandas as pd
import  numpy as np

import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import re

## Загрузка данных

In [2]:
def get_all_regions() -> List[str]:
    user_agent = fake_useragent.UserAgent()
    regions = requests.get(
        url="https://api.hh.ru/areas",
        headers={"user_agent": user_agent.random}
    ).json()
    return [str(region["id"]) for region in regions[0]["areas"]]

In [3]:
def get_count_vacancies(job_title: str, regions: List[str]) -> dict[str, dict[str, int]]:
    user_agent = fake_useragent.UserAgent()
    mean_salary_by_region = []
    result_row = {}
    for region in regions:
        url = f"https://api.hh.ru/vacancies?clusters=true&only_with_salary=true&enable_snippets=true&st=searchVacancy' \
            '&text={job_title}&search_field=name&area={region}"
        vacancies = requests.get(
            url=url,
            headers={"user_agent": user_agent.random}                    
        ).json()
        if "items" in vacancies.keys():
            for vacancy in vacancies["items"]:
                if vacancy["salary"]["from"] is None:
                    continue
                mean_salary_by_region.append(vacancy["salary"]["from"])
        else:
            continue
    result_row[job_title] = {
        "Средняя зарплата.": int(sum(mean_salary_by_region) / len(mean_salary_by_region)),
        "Количество вакансий в России.": len(mean_salary_by_region) * 43
    }
    return result_row

In [4]:
profs = pd.read_csv("../repositories/proffesions.csv")

In [24]:
options = webdriver.Firefox()
driver = webdriver.Firefox(executable_path='geckodriver')

def get_count_vacancies(job_title: str):
    URL = fr'https://hh.ru/search/vacancy?text={job_title}'

    #service = Service(executable_path='chromedriver')
    #driver = webdriver.Chrome(service=service)

    driver.get(URL)

    el = driver.find_element(By.XPATH, "//h1[@data-qa='bloko-header-3']")
    return int(''.join(re.findall(r'\d+', el.text)))

In [25]:
profs = pd.read_csv('../repositories/proffesions.csv')

In [27]:
for prof in profs[:1]["prof_name"].values:
    print(get_count_vacancies(job_title=prof))

612


In [21]:
driver = webdriver.Firefox(executable_path='./repositories/geckodriver')
oprions = webdriver.Firefox()

def get_info_by_vacancies(
        job_title: str,
        driver
    ) -> dict[str, dict[str, int]]:
    """
    Функция принимающая на вход профессию, и список всех регионов в России.
    Возвращает: {
        "job_title": {
            "mean_salary": int,
            "count_vacancies": int
    }}
    """
    def get_all_regions() -> List[str]:
        """
        Функция возвращающая, номера всех регионов в России.
        """
        user_agent = fake_useragent.UserAgent()
        regions = requests.get(
            url="https://api.hh.ru/areas",
            headers={"user_agent": user_agent.random}
        ).json()
        return [str(region["id"]) for region in regions[0]["areas"]]
    
    def get_count_vacancies(job_title: str) -> int:
        url = fr'https://hh.ru/search/vacancy?text={job_title}'
        time.sleep(3)
        driver.get(url)

        el = driver.find_element(By.XPATH, "//h1[@data-qa='bloko-header-3']")
        return int(''.join(re.findall(r'\d+', el.text)))
    
    user_agent = fake_useragent.UserAgent()
    mean_salary_by_region = []

    result_row = {}
    for region in get_all_regions():
        url = f"https://api.hh.ru/vacancies?clusters=true&only_with_salary=true&enable_snippets=true&st=searchVacancy' \
            '&text={job_title}&search_field=name&per_page=100&area={region}"
        
        vacancies = requests.get(
            url=url,
            headers={"user_agent": user_agent.random}                    
        ).json()

        for vacancy in vacancies["items"]:
            if vacancy["salary"]["from"] is None:
                continue
            mean_salary_by_region.append(vacancy["salary"]["from"])
            
    result_row[job_title] = {
        "Средняя зарплата.": int(sum(mean_salary_by_region) / len(mean_salary_by_region)),
        "Количество вакансий в России.": get_count_vacancies(job_title=job_title)
    }
    return result_row

/var/folders/ln/l54hjc713kxgxkw6rfxxgyh00000gn/T/ipykernel_72764/3979087448.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='./repositories/geckodriver')


In [22]:
get_info_by_vacancies("R-разработчик")

WebDriverException: Message: Failed to decode response from marionette
